In [2]:
import os
import yaml
import nbimporter
from zz_table_manager import TableManager
from datetime import datetime, date
# import pyspark
# from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, DateType, FloatType, TimestampType
# from pyspark.sql.functions import to_date, to_timestamp
# from lab_table_manager import TableManager
import yfinance as yf
# from lab_finalytics_database import FinalyticsDB

In [2]:
# import yaml
# import pyspark
# from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, DateType, FloatType, TimestampType

# class TableManager:
#     def __init__(self, config_file):
#         """
#         Initialize the TableManager with the path to the YAML configuration file.
#         """
#         self.config_file = config_file
#         self.config = self._load_config()

#     def _load_config(self):
#         """
#         Load the YAML configuration file.
#         """
#         with open(self.config_file, "r") as file:
#             return yaml.safe_load(file)

#     def get_schema(self, table_name):
#         """
#         Generate a PySpark schema for a given table name from the configuration.
        
#         :param table_name: Name of the table in the configuration.
#         :return: A PySpark StructType schema.
#         """
#         if table_name not in self.config["tables"]:
#             raise ValueError(f"Table '{table_name}' not found in the configuration.")

#         table_config = self.config["tables"][table_name]
#         schema = StructType([
#             StructField(field["name"], eval(field["type"])(), field["nullable"])
#             for field in table_config["schema"]
#         ])
#         return schema


In [3]:
def fetch_stock_data(symbol, start_date, end_date):
    stock = yf.Ticker(symbol)
    hist = stock.history(start=start_date, end=end_date)
    hist.reset_index(inplace=True)

    # Rename and transform columns
    hist = hist[["Date", "Open"]].rename(columns={"Date": "date", "Open": "open"})

    # Strip timezone from the 'date' column
    hist["date"] = hist["date"].dt.date  # Convert to naive date (no timezone)
    
    hist["symbol"] = symbol
    hist["import_time"] = datetime.now()
    return hist


In [4]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder \
    .appName("YFinance Data Loader") \
    .getOrCreate()

# Initialize TableManager with the config file path
config_file = "table_config.yaml"
table_manager = TableManager(config_file)

# Get schema for a specific table
table_name = "nessie.raw.stock_eod_yfinance"
schema = table_manager.get_schema(table_name)

print(schema)

symbol = "AAPL"
start_date = "2023-01-01"
end_date = "2023-12-31"
stock_data = fetch_stock_data(symbol, start_date, end_date)

stock_df = spark.createDataFrame(stock_data, schema=schema)

stock_df.show()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/25 01:20:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/25 01:20:45 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


StructType([StructField('date', DateType(), False), StructField('open', FloatType(), True), StructField('symbol', StringType(), False), StructField('import_time', TimestampType(), False)])


+----------+---------+------+--------------------+
|      date|     open|symbol|         import_time|
+----------+---------+------+--------------------+
|2023-01-03|128.92424|  AAPL|2024-12-25 01:20:...|
|2023-01-04|125.56951|  AAPL|2024-12-25 01:20:...|
|2023-01-05|125.80702|  AAPL|2024-12-25 01:20:...|
|2023-01-06|124.69868|  AAPL|2024-12-25 01:20:...|
|2023-01-09|129.11227|  AAPL|2024-12-25 01:20:...|
|2023-01-10|128.90446|  AAPL|2024-12-25 01:20:...|
|2023-01-11|129.88416|  AAPL|2024-12-25 01:20:...|
|2023-01-12|132.48679|  AAPL|2024-12-25 01:20:...|
|2023-01-13|130.65602|  AAPL|2024-12-25 01:20:...|
|2023-01-17| 133.4269|  AAPL|2024-12-25 01:20:...|
|2023-01-18| 135.3962|  AAPL|2024-12-25 01:20:...|
|2023-01-19|132.68472|  AAPL|2024-12-25 01:20:...|
|2023-01-20|133.87221|  AAPL|2024-12-25 01:20:...|
|2023-01-23|136.68265|  AAPL|2024-12-25 01:20:...|
|2023-01-24|138.84985|  AAPL|2024-12-25 01:20:...|
|2023-01-25|139.42384|  AAPL|2024-12-25 01:20:...|
|2023-01-26|141.68011|  AAPL|20

In [8]:
symbol='VFIAX'
quote = yf.Ticker(symbol)
current_date = date.today()
hist = quote.history(start='2024-12-21', end=current_date)

# Reset index to include Date as a column and format it
hist.reset_index(inplace=True)
hist["Date"] = hist["Date"].dt.date

print(hist)

# limit and stablize the fields of hist
hist = hist[['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']]


         Date        Open        High         Low       Close  Volume  \
0  2024-12-23  551.150024  551.150024  551.150024  551.150024       0   
1  2024-12-24  557.229980  557.229980  557.229980  557.229980       0   

   Dividends  Stock Splits  Capital Gains  
0        0.0           0.0            0.0  
1        0.0           0.0            0.0  


In [4]:
print(hist)

         Date        Open        High         Low       Close   Volume  \
0  2024-12-23  173.639999  177.570007  170.779999  177.110001  7031100   
1  2024-12-24  178.070007  182.750000  177.570007  182.639999  2183200   

   Dividends  Stock Splits  
0      0.321           0.0  
1      0.000           0.0  
